# Simple Policy Gradient

This notebook contains the implementation of the Simple Policy Gradient Algorithm using TensorFlow.
<br/>
This notebook is created while going through the official Spinning up in Deep RL Docs.

In [1]:
# Required modules
!pip install gym

    100% |████████████████████████████████| 1.5MB 13.4MB/s 
    100% |████████████████████████████████| 1.0MB 18.0MB/s 
  Running setup.py bdist_wheel for gym ... - \ | / done
  Stored in directory: /root/.cache/pip/wheels/6c/3a/0e/b86dee98876bb56cdb482cc1f72201035e46d1baf69d10d028
Successfully built gym


In [0]:
# Import required modules
import numpy as np
import tensorflow as tf
import gym
from gym.spaces import Discrete, Box

In [0]:
# Arguments
env_name = 'CartPole-v0'
render = True

In [4]:
# Create the env
env = gym.make('CartPole-v0')

/usr/local/lib/python3.6/dist-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


In [5]:
# Get the action space size and observation space size
act_size = env.action_space.n
obs_size = env.observation_space.shape[0]

print ('Action Space Size: {}'.format(act_size),
       '\nObservation Space Size: {}'.format(obs_size))

Action Space Size: 2 
Observation Space Size: 4


In [0]:
# Network Hyperparameters
layers = 2
hneurons = [32, act_size]
epochs = 50
batch_size = 5000
lr = 1e-2
hid_act = tf.tanh
out_act = None

In [0]:
# Build the network
inp = tf.placeholder(shape=(None, obs_size), dtype=tf.float32, name='input')

a1 = tf.layers.dense(inp, units=hneurons[0], activation=hid_act)
logits = tf.layers.dense(a1, units=hneurons[1], activation=None)

# Select the action
actions = tf.squeeze(tf.multinomial(logits=logits, num_samples=1), axis=1)

# Loss function whose gradient is the policy gradient
wplaceholder = tf.placeholder(shape=(None,), dtype=tf.float32)
act_ph = tf.placeholder(shape=(None,), dtype=tf.int32)
action_masks = tf.one_hot(act_ph, act_size)
log_probs = tf.reduce_sum(action_masks * tf.nn.log_softmax(logits), axis=1)
loss = -tf.reduce_mean(wplaceholder * log_probs)

# Make the train op
train_op = tf.train.AdamOptimizer(learning_rate=lr).minimize(loss)

In [12]:
# Training loop
for epoch in range(epochs):
    batch_loss, batch_rets, batch_lens = train_one_epoch()
    print ('Epoch: {:.3f} Loss: {:.3f} Return: {:.3f} ep_len: {:.3f}'
           .format(epoch+1, batch_loss, np.mean(batch_rets), np.mean(batch_lens)))

NameError: ignored

In [10]:
# TensorBoard Setup
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('tensorboard --logdir=./tboard/FrozenLake-v0/ &')
get_ipython().system_raw('./ngrok http 6006 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2018-12-01 19:11:33--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 34.232.181.106, 34.226.180.131, 34.232.40.183, ...
Connecting to bin.equinox.io (bin.equinox.io)|34.232.181.106|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5363700 (5.1M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]   5.11M  10.6MB/s    in 0.5s    

2018-12-01 19:11:33 (10.6 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [5363700/5363700]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
http://d2cfdd99.ngrok.io
